# **Homework 1: Linear Regression**

## **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [1]:
import sys
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv('./data/train.csv', encoding = 'big5')
train.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日期      4320 non-null   object
 1   測站      4320 non-null   object
 2   測項      4320 non-null   object
 3   0       4320 non-null   object
 4   1       4320 non-null   object
 5   2       4320 non-null   object
 6   3       4320 non-null   object
 7   4       4320 non-null   object
 8   5       4320 non-null   object
 9   6       4320 non-null   object
 10  7       4320 non-null   object
 11  8       4320 non-null   object
 12  9       4320 non-null   object
 13  10      4320 non-null   object
 14  11      4320 non-null   object
 15  12      4320 non-null   object
 16  13      4320 non-null   object
 17  14      4320 non-null   object
 18  15      4320 non-null   object
 19  16      4320 non-null   object
 20  17      4320 non-null   object
 21  18      4320 non-null   object
 22  19      4320 non-null   

In [4]:
len(train['日期'].unique())

240

In [5]:
len(train['測項'].unique())

18

In [6]:
print('Any null value in dataFrame? {}'.format(train.isnull().any().any()))

Any null value in dataFrame? False


## **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [7]:
data = train.sort_values(['日期', '測項'], ascending=[True, True])

data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
raw_data.shape

(4320, 24)

# **Extract Features (1)**
![圖片說明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![圖片說明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [30]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [51]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x.shape)
print(y.shape)

(5652, 162)
(5652, 1)


# **Normalize (1)**


In [47]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

array([[-1.35831125, -1.3581189 , -1.3580378 , ...,  0.26554626,
         0.26582281, -1.14150665],
       [-1.35831125, -1.3581189 , -1.51692754, ...,  0.26554626,
        -1.1412987 , -1.32915977],
       [-1.35831125, -1.51711358, -1.67581727, ..., -1.14213286,
        -1.3289149 , -0.86002698],
       ...,
       [ 1.34694223,  1.34479058,  1.34308777, ..., -0.95444231,
         0.07820661,  0.0782386 ],
       [ 1.34694223,  1.34479058,  1.34308777, ...,  0.07785571,
         0.07820661, -0.39089419],
       [ 1.34694223,  1.34479058,  1.18419803, ...,  0.07785571,
        -0.39083389, -0.20324107]])

#**Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中 report 的第二題、第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [48]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print('x_train_set shape: {}'.format(x_train_set.shape))
print('y_train_set shape: {}'.format(y_train_set.shape))
print('x_validation shape: {}'.format(x_validation.shape))
print('y_validation shape: {}'.format(y_validation.shape))

x_train_set shape: (4521, 162)
y_train_set shape: (4521, 1)
x_validation shape: (1131, 162)
y_validation shape: (1131, 1)


# **Training**
![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [55]:
dim = 18 * 9 + 1 # 18 features * 9 days + 1 bias
w = np.zeros([dim, 1])
bias = np.ones([12 * 471, 1]) # 1 * bias (w last element) = bias
X = np.concatenate((bias, x), axis = 1).astype(float)
print(x.shape, X.shape, w.shape)

learning_rate = 100
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 10 ** -10
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(X, w) - y, 2))/471/12)#rmse
    if(t%1000==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(X.transpose(), np.dot(X, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)

(5652, 162) (5652, 163) (163, 1)
0:27.071214829194115
1000:7.2126804901956625
2000:6.710533512556686
3000:6.486086931251134
4000:6.358626944972734
5000:6.27655399190085
6000:6.219432785043238
7000:6.177480512559558
8000:6.145411825108761
9000:6.120126315313074


array([[ 2.55476649e-02],
       [-9.74346447e-03],
       [-1.12377067e-02],
       [-1.13945520e-02],
       [-1.19255315e-02],
       [-4.27023748e-03],
       [-2.86049605e-03],
       [ 4.99775050e-03],
       [ 1.18883812e-02],
       [ 2.27723827e-02],
       [ 2.48503955e-02],
       [ 2.64736139e-02],
       [ 2.75453976e-02],
       [ 2.23801626e-02],
       [ 3.19083440e-02],
       [ 4.13930226e-02],
       [ 4.00918098e-02],
       [ 4.53650546e-02],
       [ 7.16598733e-02],
       [ 3.95905912e-02],
       [-7.45956301e-03],
       [ 3.98054310e-02],
       [-3.94233059e-02],
       [ 1.08707210e-01],
       [-3.47888585e-02],
       [-3.73454878e-02],
       [ 2.31861214e-01],
       [ 5.59030603e-01],
       [-1.03688606e-01],
       [-6.86528169e-02],
       [-6.37199564e-02],
       [-2.01280399e-03],
       [ 1.10438824e-01],
       [ 4.74253382e-02],
       [ 7.94943531e-03],
       [ 7.92002249e-02],
       [ 2.55679037e-01],
       [ 7.45021225e-02],
       [ 2.4

# **Testing**
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [56]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
test = pd.read_csv('./data/test.csv', header = None, encoding = 'big5')
test.head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [57]:
test_data = test.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/Library/Python/3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Python/3.7/site-packages/pandas/core/frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24438335, -0.24515617, ..., -0.67290649,
        -1.0474906 ,  0.0782386 ],
       [ 1.        , -1.35831125, -1.51711358, ...,  0.17170099,
        -0.10940959, -0.48472075],
       [ 1.        ,  1.50607478,  1.34479058, ..., -1.32982341,
        -1.0474906 , -0.5785473 ],
       ...,
       [ 1.        ,  0.39214688,  0.5498172 , ...,  0.26554626,
        -0.20321769,  1.20415729],
       [ 1.        , -1.83570892, -1.83510293, ..., -1.04828759,
        -1.1412987 , -1.14150665],
       [ 1.        , -1.35831125, -1.3581189 , ...,  2.98705924,
         3.26768203,  1.76711664]])

# **Prediction**
說明圖同上

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

有了 weight 和測試資料即可預測 target。

In [60]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[-9.71860418e-01],
       [ 3.69910098e-01],
       [ 2.34357960e-01],
       [-1.78197354e+00],
       [-2.15045450e-01],
       [-3.80236391e-01],
       [ 6.24389232e-01],
       [ 1.24123416e+00],
       [-4.31874187e-01],
       [ 3.59143411e+00],
       [-3.60934356e-02],
       [-2.45093573e+00],
       [ 2.91732467e+00],
       [ 4.20012560e+00],
       [-1.07097639e+00],
       [-2.07011556e+00],
       [ 6.37144781e-01],
       [ 4.29547173e+00],
       [-3.39522477e+00],
       [-1.39274563e+00],
       [ 3.24205991e+00],
       [ 4.34835406e+00],
       [-1.30162951e+00],
       [-3.99810144e-01],
       [-7.14918687e-01],
       [ 1.79895223e+00],
       [-2.85891545e-01],
       [ 5.02697690e+00],
       [-1.39651912e+00],
       [ 4.59011129e+00],
       [ 7.24259886e-02],
       [-1.71765462e+00],
       [ 5.23955675e-01],
       [-4.45477950e-01],
       [ 8.81798472e-01],
       [ 1.50560463e+00],
       [ 2.66897096e+00],
       [ 1.99897206e+00],
       [ 2.5

# **Save Prediction to CSV File**


In [61]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', -0.9718604177104275]
['id_1', 0.3699100975035335]
['id_2', 0.23435796037247508]
['id_3', -1.7819735407341764]
['id_4', -0.21504544975753284]
['id_5', -0.3802363907903797]
['id_6', 0.6243892321570776]
['id_7', 1.2412341566797114]
['id_8', -0.43187418669974376]
['id_9', 3.591434109740299]
['id_10', -0.0360934355538318]
['id_11', -2.450935730903991]
['id_12', 2.917324672941394]
['id_13', 4.200125596190397]
['id_14', -1.0709763864073065]
['id_15', -2.070115558954504]
['id_16', 0.6371447811596999]
['id_17', 4.295471727946587]
['id_18', -3.395224770101555]
['id_19', -1.392745634470393]
['id_20', 3.2420599121326807]
['id_21', 4.348354057963984]
['id_22', -1.3016295145846386]
['id_23', -0.39981014409172955]
['id_24', -0.7149186865857777]
['id_25', 1.7989522300020058]
['id_26', -0.28589154517541393]
['id_27', 5.026976904515582]
['id_28', -1.3965191216158774]
['id_29', 4.590111291687826]
['id_30', 0.07242598863419174]
['id_31', -1.7176546181967807]
['id_32', 0.5239556751

相關 reference 可以參考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是為了看一下資料和結果的呈現，拿掉也無妨。另外，在自己的 linux 系統，可以將檔案寫死的的部分換成 sys.argv 的使用 (可在 terminal 自行輸入檔案和檔案位置)。

最後，可以藉由調整 learning rate、iter_time (iteration 次數)、取用 features 的多寡(取幾個小時，取哪些特徵欄位)，甚至是不同的 model 來超越 baseline。

Report 的問題模板請參照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit